In [17]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# from skimage.feature import hog
from lesson_functions import *
%matplotlib inline

### Read in dataset

In [11]:
# Read in cars and notcars
image_files = glob.glob('./training_data/smallset/*/*.jpeg')
car_files = []
notcar_files = []
for fname in image_files:
    if 'image' in fname or 'extra' in fname:
        notcar_files.append(fname)
    else:
        car_files.append(fname)

# Reduce the sample size because
# The quiz evaluator times out after 13s of CPU time
sample_size = 500
car_files = car_files[0:sample_size]
notcar_files = notcar_files[0:sample_size]

### Set parameters and train model

In [18]:
#---------------------
# Training parameters
#---------------------

colorspace = 'HSV' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb

# spatial parameter
spatial = (14,14) # 14 seems optimal

# histogram parameters
histbin = 64 # 64 seems pretty good

# HOG parameters
orient = 12
pix_per_cell = 8
cell_per_block = 2
hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"

#--------------------
# Feature extraction
#--------------------

t=time.time()
car_features = extract_features(car_files, color_space=colorspace,
                                spatial_size=spatial, hist_bins=histbin,
                                orient=orient, pix_per_cell=pix_per_cell,
                                cell_per_block=cell_per_block, hog_channel=hog_channel)
notcar_features = extract_features(notcar_files, color_space=colorspace,
                                   spatial_size=spatial, hist_bins=histbin,
                                   orient=orient, pix_per_cell=pix_per_cell,
                                   cell_per_block=cell_per_block, hog_channel=hog_channel)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')

#------------------
# Setup X & y data
#------------------

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
X_scaler = StandardScaler().fit(X)
scaled_X = X_scaler.transform(X)
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

#------------------
# Train/Test split
#------------------

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

# Use a linear SVC 
svc = LinearSVC()

#-------------
# Train model
#-------------

# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')

# Check the score of the SVC
print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))

# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print('My SVC predicts: ', svc.predict(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')

8.26 Seconds to extract HOG features...
Using: 12 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 7836
0.23 Seconds to train SVC...
Test Accuracy of SVC =  1.0
My SVC predicts:  [ 0.  0.  0.  1.  1.  0.  0.  0.  0.  1.]
For these 10 labels:  [ 0.  0.  0.  1.  1.  0.  0.  0.  0.  1.]
0.00233 Seconds to predict 10 labels with SVC


In [ ]:
dist_pickle = pickle.load( open("svc_pickle.p", "rb" ) )
svc = dist_pickle["svc"] #SVM-Classifier
X_scaler = dist_pickle["scaler"]
orient = 9 # num of HOG orientations
pix_per_cell = 8 
cell_per_block = 8
spatial_size = 
hist_bins = 

In [4]:
img = mpimg.imread('./test_images/test1.jpg')

In [ ]:
ystart = 400
ystop = 656
scale = 1.5
    
out_img = find_cars(img, ystart, ystop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins)

plt.imshow(out_img)